In [1]:
import os 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
import os
from PIL import Image
from os import listdir
import cv2
%matplotlib inline



In [3]:
ORIG_INPUT_DATASET = "baris/O/"
BASE_PATH = "dataset"

# Load in the images
instances = []
for filepath in os.listdir(ORIG_INPUT_DATASET):
    instances.append(cv2.imread("baris/O/{}".format(filepath)))



### Extracting only faces of objects using cascade classifier 

In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
count = 0
path = 'baris/croped'
for i in instances:
    gray = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)

    face = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for (x,y,w,h) in face:
        cv2.rectangle(face, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite(str(path) +'/' + str(count) + ".jpg", i[y:y+h,x:x+w])


        

In [6]:
count = 0
face_path = 'baris/croped'
resized_IMG_path = 'baris/resized/'
for img  in os.listdir(face_path):
    count += 1
    file = Image.open("baris/croped/{}".format(img))
    img_inter = file.resize((250, 250))
    img_inter.save('baris/resized/{}.jpg'.format(count))


In [7]:
img_height, img_width = 250, 250

base_model = ResNet50(weights = 'imagenet',
                      include_top = False, 
                      input_shape = (img_height, img_width, 3))


In [8]:
model_without_top = ResNet50(weights = 'imagenet',
                             include_top = False, 
                             input_shape = (img_height, img_width, 3))

                              
model_with_top = ResNet50(weights= 'imagenet',
                          include_top = True, 
                          input_shape = (224, 224, 3))



In [ ]:
model_without_top.summary()

In [ ]:
model_with_top.summary()

I labeled 246 images as target values manually and saved them in labeles.txt file. 
Note: I want to detect Baris Ozcan's (a famous Youtuber) face while he is talking in his videos.

In [11]:
with open("labels.txt", 'r') as f:
    manual_labels = f.read()

manual_labels = manual_labels.replace(',', '')    
labels = [i for i in manual_labels]
len(labels)

246

In [12]:
# Important to sort the files in the order in which we see them in folder
directory = "data"
files = os.listdir(directory)
files.sort(key=lambda f: int(f.split('.')[0]))

In [14]:
trainX, testX, trainY, testY =  train_test_split(files[:len(labels)], labels, stratify=labels, train_size=0.2)

In [17]:
classes = ['0', '1']
num_classes = len(classes)

for layer in base_model.layers:
    layer.trainable = False

global_average_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(num_classes, activation = 'sigmoid')(global_average_pooling)

face_classifier = keras.models.Model(inputs = base_model.input, outputs = output, name = 'ResNet50')

# saving classifier 
